In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
warnings.filterwarnings('ignore',message='profile')
warnings.filterwarnings('ignore',message='_astropy_init')
warnings.filterwarnings('ignore',message='ConfigurationDefaultMissingWarning ')

import os
import sys

In [4]:
from astroquery.vizier import Vizier
import astropy.units as u
import astropy.coordinates as coord
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle

#import pyvo as vo

In [5]:
Vizier.ucd  = 'phot.flux.density;em.radio.*'

In [6]:

RA = raw_input("Enter RA of target hh mm ss ")
DEC = raw_input("Enter Dec of target dd mm ss ")

Enter RA of target hh mm ss 19 54 30
Enter Dec of target dd mm ss -30 25 15


In [7]:
MyRadius = raw_input('Enter rsearch radius e.g. 16m, 16d, 16s ')

Enter rsearch radius e.g. 16m, 16d, 16s 1d


In [8]:
def InputCoord(arg1,arg2):
    '''This takes a user input of the RA and Dec and then 
    convert it to RaTarget and DecTarget in deg for use in the VO search.
    Input: arg1(RA), arg2(Dec) NB: They are str in this case.
    Output: RaTarget,DecTarget
    '''
    hh = float(arg1.split(' ')[0])
    mm = float(arg1.split(' ')[1])/60.
    ss = float(arg1.split(' ')[2])/3600.

    dec_dd = float(arg2.split(' ')[0])
    dec_mm = float(arg2.split(' ')[1])/60.
    dec_ss = float(arg2.split(' ')[2])/3600.

    if dec_dd <0:
        dec = dec_dd-dec_mm-dec_ss
    else:
        dec = dec_dd+dec_mm+dec_ss

    RaTarget = (hh+mm+ss)*15.
    DecTarget = dec
    
    return RaTarget,DecTarget

In [9]:
Ra,Dec = InputCoord(RA, DEC)

In [10]:
Dec 

-30.420833333333334

In [11]:
# The commonly used Radio catalogues on Vizier is given by:
# first_cat(VIII/92/first14)   
# nvss_cat(VIII/65/nvss)
# SUMSS_cat(VIII/81B/sumss212)


if (Dec > -90.) and (Dec < -30.):
    print("Recommend one use SUMSS survey ")
    catalog_list = Vizier.find_catalogs('VIII/81B/sumss212') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())

elif (Dec > -30.) and (Dec < 90.):
    print("Recommend to use NVSS or FIRST if Dec >-10 ")
    catalog_list = Vizier.find_catalogs('VIII/65/nvss') 

    result = Vizier.query_region(coord.SkyCoord(ra=Ra, dec=Dec,unit=(u.deg, u.deg),frame='icrs')
                                 ,radius=MyRadius,catalog=catalog_list.keys())
else:
    print("Check your input ")


/usr/local/lib/python2.7/site-packages/astroquery/vizier/core.py:567: UserWarning: VOTABLE parsing raised exception: 
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


Recommend one use SUMSS survey 


In [12]:
# Printing catalogues that are accessible
for i in range(len(result)):
    print i,'-',result[i].meta['description']

0 - Main catalogue
1 - Quasars (M_v_<-22.25), BL Lac and AGNs (M_v_>-22.25)
2 - The Catalogue
3 - Catalog Data
4 - Zenith Survey (-37 to -29)
5 - The Texas Survey
6 - The sources in the simpler model.
7 - The NVSS Catalog
8 - HI line data for 4315 extragalactic sources
9 - Properties of detected galaxies derived from spectra in digital archive
10 - The SUMSS Catalog, Version 2.1r (Feb-16-2012)
11 - GLEAM EGC catalog, version 2
12 - The 1RXS catalogue
13 - 1RXS Correlation entries
14 - 1RXS Correlation to NED
15 - 1RXS Correlation to NVSS, FIRST
16 - 1RXS Correlation to PRI, RITTER, SIMBAD, TYCHO, EUVE
17 - 1RXS Correlation to HST_GSC
18 - NVSS rotation measure catalog
19 - Observing times of NVSS sources
20 - The catalog of 2097 objects detected at 5, 7 or 9GHz subbands
21 - Associations of 3FGL objects with compact radio sources
22 - Structure of Southern Extended Extragalactic Radio Sources
23 - RBSC-NVSS Extragalactic Identifications
24 - The CRATES catalog
25 - Quasars LQAC-2
26 - 

In [13]:
indx = int(raw_input("Enter catalogue number to use "))

Enter catalogue number to use 10


In [14]:
MyCat = result[result.keys()[indx]]
print "Catalogue chosen is: ", result[indx].meta['description']
MyCat[:5]

Catalogue chosen is:  The SUMSS Catalog, Version 2.1r (Feb-16-2012)


_RAJ2000,_DEJ2000,_r,RAJ2000,e_RAJ2000,DEJ2000,e_DEJ2000,Sp,e_Sp,St,e_St,MajAxis,MinAxis,PA,dMajAxis,dMinAxis,dPA,Mosaic,Nm,Xpos,Ypos
deg,deg,deg,"""h:m:s""",arcs,"""d:m:s""",arcs,mJy,mJy,mJy,mJy,arcs,arcs,deg,arcs,arcs,deg,,,pix,pix
float64,float64,float64,str11,float32,str12,float32,float32,float32,float64,float32,float32,float32,float32,float32,float32,float32,str8,uint8,float32,float32
297.5591667,-30.1476389,0.960072,19 50 14.20,11.9,-30 08 51.50,11.6,14.2,3.5,21.0,5.2,84.9,77.5,126.6,0.0,0.0,--,J1948M32,1,546.8,695.0
297.5619583,-30.3506667,0.919704,19 50 14.87,7.1,-30 21 02.40,4.0,15.5,2.9,15.5,2.9,84.9,45.1,92.8,0.0,0.0,--,J1948M32,1,546.3,659.8
297.6094167,-29.9482778,0.996982,19 50 26.26,2.4,-29 56 53.80,3.4,42.8,3.4,42.8,3.4,84.9,51.5,168.6,0.0,0.0,--,J1948M32,1,532.2,729.5
297.6441667,-30.3703611,0.847527,19 50 34.60,5.3,-30 22 13.30,3.7,22.9,3.1,24.7,3.3,84.9,50.3,107.5,0.0,0.0,--,J1948M32,1,523.1,656.3
297.7110417,-30.6063889,0.808955,19 50 50.65,8.8,-30 36 23.00,5.2,11.3,2.7,11.3,2.7,84.9,45.4,80.3,0.0,0.0,--,J1948M32,1,504.7,615.3


In [15]:
if "SUMSS" in result[indx].meta['description']:
    CatChosen = 'SUMSS'
elif "NVSS" in result[indx].meta['description']:
    CatChosen = 'NVSS'
else:
    print("Non statndard cat chosesn may break code")

In [16]:
# LSM should be of the following format:
# #name ra_d dec_d i q u v emaj_s emin_s pa_d

MyLSM = []
if CatChosen == 'SUMSS':
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['Sp'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))
        
elif CatChosen == 'NVSS':
    
    for Indx in range(len(MyCat)):
        c = SkyCoord(MyCat['RAJ2000'][Indx]+' '+MyCat['DEJ2000'][Indx], unit=(u.hourangle, u.deg))

        ra_d = c.ra.deg
        dec_d = c.dec.deg

        # i should be in Jansky, Q, U and V will be 0 for now 

        StokesI = MyCat['S1.4'][Indx]*1.e-3
        StokesQ = 0.0
        StokesU = 0.0
        StokesV = 0.0

        emaj_s = Angle(MyCat['MajAxis'][Indx],unit = u.arcsec)  
        emin_s = Angle(MyCat['MinAxis'][Indx],unit = u.arcsec)  
        pa_d = Angle(MyCat['PA'][Indx],unit = u.deg)

        MyLSM.append((Indx, ra_d, dec_d, StokesI, StokesQ, StokesU, StokesV, emaj_s.deg, emin_s.deg, pa_d.deg ))

In [17]:
MyLSMHeader = '#format:name ra_d dec_d i q u v emaj_s emin_s pa_d'

In [18]:
np.savetxt('test_LSM.lsm', np.array(MyLSM),header=MyLSMHeader,comments='',
           fmt= '%i %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e %.5e')

In [19]:
# Copy the sky model to the path you want